# Part 2: Homework 2
*Course: Mathematics 2 (DataScience@FRI, University of Ljubljana)*

In [7]:
# Global imports
import time
import subprocess
import random
import numpy as np
import scipy
import networkx as nx
from cvxopt import solvers, matrix

from optimizers.NelderMeadOptimizer import NelderMeadOptimizer
from optimizers.MaximalWeightMatchingLocalSearch import MaximalWeightMatchingLocalSearch, sum_graph_weights

In [2]:
# Seed
random.seed(0)

## Nelder Mead Method

In this section we test our implementation of the Nelder Mead method. We test it on different diameters used for forming a tetrahedron and on 3 different functions.

In [2]:
possible_diameters = [1, 3, 5]

### Function 1

In [3]:
def cost_function_1(sample_points):
    x = sample_points[0]
    y = sample_points[1]
    z = sample_points[2]

    return (x - z) ** 2 + (2 * y + z) ** 2 + (4 * x - 2 * y + z) ** 2 + x + y

In [4]:
x_origin = np.zeros(3)
x_true = np.array([-1 / 6, -11 / 48, 1 / 6])
for diameter in possible_diameters:
    print(f"Diameter: {diameter}")
    nm_optimizer = NelderMeadOptimizer(x_origin=x_origin,
                                       diameter=diameter,
                                       f_cost=cost_function_1,
                                       x_true=x_true,
                                       max_iter=10000)
    nm_optimizer.minimize()

Diameter: 1
Solution: x=[-0.16686424 -0.22943509  0.16662862], y=-0.1979162257488882
Abs. diff. to true x: [1.97576083e-04 2.68421374e-04 3.80418885e-05]
Abs. diff. to true y: 4.409177784825413e-07
Time spent: 0.003686189651489258, Steps: 44
Diameter: 3
Solution: x=[-0.16682548 -0.22950194  0.16678644], y=-0.19791626166819298
Abs. diff. to true x: [0.00015881 0.00033527 0.00011978]
Abs. diff. to true y: 4.049984737020118e-07
Time spent: 0.0024607181549072266, Steps: 55
Diameter: 5
Solution: x=[-0.16646306 -0.22897428  0.16663081], y=-0.1979163325152108
Abs. diff. to true x: [2.03605282e-04 1.92390608e-04 3.58607933e-05]
Abs. diff. to true y: 3.34151455894105e-07
Time spent: 0.002635955810546875, Steps: 67


### Function 2

In [5]:
def cost_function_2(sample_points):
    x = sample_points[0]
    y = sample_points[1]
    z = sample_points[2]

    return (x - 1) ** 2 + (y - 1) ** 2 + 100 * (y - x ** 2) ** 2 + 100 * (z - y ** 2) ** 2

In [6]:
x_origin = np.array([-1, 1.2, 1.2])
x_true = np.array([1, 1, 1])
for diameter in possible_diameters:
    print(f"Diameter: {diameter}")
    nm_optimizer = NelderMeadOptimizer(x_origin=x_origin,
                                       diameter=diameter,
                                       f_cost=cost_function_2,
                                       x_true=x_true,
                                       max_iter=10000)
    nm_optimizer.minimize()


Diameter: 1
Solution: x=[0.9999845  0.99996129 0.99986846], y=3.0069944518989106e-07
Abs. diff. to true x: [1.54996014e-05 3.87054993e-05 1.31541009e-04]
Abs. diff. to true y: 3.0069944518989106e-07
Time spent: 0.010965108871459961, Steps: 192
Diameter: 3
Solution: x=[1.00018791 1.00040822 1.00085098], y=4.248894201940119e-07
Abs. diff. to true x: [0.00018791 0.00040822 0.00085098]
Abs. diff. to true y: 4.248894201940119e-07
Time spent: 0.008292675018310547, Steps: 209
Diameter: 5
Solution: x=[1.0000865  1.00014676 1.00032761], y=2.1393312362123425e-07
Abs. diff. to true x: [8.65017793e-05 1.46763219e-04 3.27609144e-04]
Abs. diff. to true y: 2.1393312362123425e-07
Time spent: 0.00533294677734375, Steps: 146


### Function 3

In [7]:
def cost_function_3(sample_points):
    x = sample_points[0]
    y = sample_points[1]

    return (1.5 - x + x * y) ** 2 + (2.25 - x + x * y ** 2) ** 2 + (2.625 - x + x * y ** 3) ** 2

In [8]:
x_origin = np.array([4.5, 4.5])
x_true = np.array([3, 0.5])
for diameter in possible_diameters:
    print(f"Diameter: {diameter}")
    nm_optimizer = NelderMeadOptimizer(x_origin=x_origin,
                                       diameter=diameter,
                                       f_cost=cost_function_3,
                                       x_true=x_true,
                                       max_iter=10000)
    nm_optimizer.minimize()

Diameter: 1
Solution: x=[3.00194849 0.50034446], y=1.04863522008143e-06
Abs. diff. to true x: [0.00194849 0.00034446]
Abs. diff. to true y: 1.04863522008143e-06
Time spent: 0.0017240047454833984, Steps: 34
Diameter: 3
Solution: x=[2.99976302 0.49988583], y=7.974757109495762e-08
Abs. diff. to true x: [0.00023698 0.00011417]
Abs. diff. to true y: 7.974757109495762e-08
Time spent: 0.0024950504302978516, Steps: 36
Diameter: 5
Solution: x=[2.99884709 0.49979708], y=3.7206718438870837e-07
Abs. diff. to true x: [0.00115291 0.00020292]
Abs. diff. to true y: 3.7206718438870837e-07
Time spent: 0.002170085906982422, Steps: 50


## Black Box

In this section we optimize the given black box model using our own implementation of the Nelder Mead method and compare the results with a commercial L-BFGS-B optimizer.

In [10]:
student_code = "63170327"
x_origin = np.array([0, 0, 0])

for i in range(3):
    print(f"Function {i + 1}")
    print(f"Nelder Mead")


    def black_box_optimization(sample_points):
        x = sample_points[0]
        y = sample_points[1]
        z = sample_points[2]
        ix_func = str(i + 1)
        result = subprocess.run(['./hw4_mac', student_code, ix_func, str(x), str(y), str(z)],
                                stdout=subprocess.PIPE)
        return float(result.stdout)


    nm_optimizer = NelderMeadOptimizer(x_origin=x_origin,
                                       diameter=1,
                                       f_cost=black_box_optimization,
                                       max_iter=200)
    nm_optimizer.minimize()

    print(f"L-BFGS-B")
    start_time = time.time()
    result = scipy.optimize.minimize(x0=x_origin, fun=black_box_optimization)
    total_time = time.time() - start_time

    print(f"Solution: x={result.x}, y={black_box_optimization(result.x)}")
    print(f"Time spent: {total_time}")

Function 1
Nelder Mead
Solution: x=[0.72302797 0.30711387 0.71356174], y=0.723072000026685
Time spent: 1494.4380600452423, Steps: 76
L-BFGS-B
Solution: x=[0.72300518 0.30709772 0.7135977 ], y=0.723071360008897
Time spent: 376.8282527923584
Function 2
Nelder Mead
Solution: x=[0.72327655 0.30775949 0.71251399], y=0.723071738886136
Time spent: 611.3038651943207, Steps: 35
L-BFGS-B
Solution: x=[0.72300002 0.30709999 0.71360008], y=0.723071360000003
Time spent: 50.631964921951294
Function 3
Nelder Mead
Solution: x=[0.72239463 0.30655337 0.71348252], y=0.723072039071156
Time spent: 645.1453537940979, Steps: 38
L-BFGS-B
Solution: x=[0.72300005 0.30710004 0.71359988], y=0.72307136000002
Time spent: 50.6066210269928


## Local Search Study: Maximal Weight Matching

In this section we show how local search can be used to solve the maximal weight matching problem. When finding the next best solution in every iteration we produced matchings that are k-adjacent. We test different k: 1, 2, 3.

In [3]:
dim = 20
G = nx.grid_2d_graph(dim, dim)
w = []
for source, target in G.edges:
    runif_1_2 = random.uniform(1, 2)
    G[source][target]["weight"] = runif_1_2
    w.append(runif_1_2)

In [8]:
# Using a commercial solver to solve the maximal weight matching problem
M_opt_edges = nx.max_weight_matching(G=G)
M_opt = nx.edge_subgraph(G=G, edges=M_opt_edges)
M_opt_sum = sum_graph_weights(graph=M_opt)
print(f"Optimal solution: {M_opt_sum}")

Optimal solution: 339.28663641100064


In [ ]:
M_opt_edges = nx.max_weight_matching(G=G)
M_opt = nx.edge_subgraph(G=G, edges=M_opt_edges)
for source, target in M_opt_edges:
    M_opt[source][target]["weight"] = G[source][target]["weight"]
M_opt_sum = sum_graph_weights(graph=M_opt)


In [4]:
for k in [1, 2, 3]:
    print(f"k={k}")
    ls_optimizer = MaximalWeightMatchingLocalSearch(G=G,
                                                    k=k)
    ls_optimizer.optimize()

k=1


100%|██████████| 10000/10000 [00:14<00:00, 697.91it/s]


Sum of weights=205.2433331662313
Time spent: 14.34494686126709, Steps: 10000
k=2


100%|██████████| 10000/10000 [00:20<00:00, 498.55it/s]


Sum of weights=192.64855780870118
Time spent: 20.059540033340454, Steps: 10000
k=3


100%|██████████| 10000/10000 [00:25<00:00, 395.46it/s]

Sum of weights=193.36573082813828
Time spent: 25.2884840965271, Steps: 10000


In [5]:
for k in [1, 2, 3]:
    print(f"k={k}")
    ls_optimizer = MaximalWeightMatchingLocalSearch(G=G,
                                                    k=k,
                                                    optimized_edge_picking=True)
    ls_optimizer.optimize()

k=1


100%|██████████| 10000/10000 [07:31<00:00, 22.16it/s]


Sum of weights=203.88814131587904
Time spent: 451.3342270851135, Steps: 10000
k=2


100%|██████████| 10000/10000 [15:18<00:00, 10.88it/s]


Sum of weights=207.1449337978389
Time spent: 918.94389295578, Steps: 10000
k=3


100%|██████████| 10000/10000 [22:07<00:00,  7.53it/s]

Sum of weights=179.1420475692353
Time spent: 1327.9340589046478, Steps: 10000


### Relax Maximal Weight Matching problem

In [6]:
# Given graph incidence matrix
A = nx.incidence_matrix(G=G).todense()
# Upper and lower bounds for e that belongs to [0,1]^E
A = np.concatenate([A, np.eye(len(G.edges))], axis=0)
A = np.concatenate((A, -np.eye(len(G.edges))), axis=0)

# This follows from x = 1 if the edge is in matching M and x = 0 otherwise!
b = np.ones((dim ** 2, 1))
# We have upper and lower bound constraints , because x belongs to [0,1]^E
# Upper bound constraints
b = np.concatenate((b, np.ones((len(G.edges), 1))), axis=0)
# Lower bound constraints
b = np.concatenate((b, np.zeros((len(G.edges), 1))), axis=0)

# Optimizing for weights, where we are trying to maximize weight sum or equivalently minimize the negative weights sum
c = - np.array(w)

# Solver minimizes c^T @ x, given Ax<=b
solvers.options['show_progress'] = False
solution = solvers.lp(c=matrix(c),
                      G=matrix(A),
                      h=matrix(b))

print(f"Cost: {abs(solution['primal objective'])}")

/var/folders/cn/hhym6n8j6bv_xfr8rvgvz4k80000gn/T/ipykernel_20409/1552239093.py:2: FutureWarning: incidence_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.incidence_matrix(G=G).todense()


Cost: 339.2866355197331
